## AB Testing Meli notebook

### importando librerias

In [2]:
# Preparamos el entorno
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt


import sys
import os
module_path = os.path.abspath(os.path.join('/Users/juanmanuelpaiba/Documents/Juan_Paiba/AB_Testing_Meli/', 'python_scripts'))
if module_path not in sys.path:
    sys.path.append(module_path)
import utilities

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# Cargamos los datos y creamos el dataframe a utilizar
df = pd.read_csv('/Users/juanmanuelpaiba/Documents/Juan_Paiba/AB_Testing_Meli/data/Inputs/Experiments DataSet For Excercise-small.csv', sep=",")
df.head()

,event_name,item_id,timestamp,site,experiments,user_id
0,SEARCH,NaN,2021-08-02T23:55:38.966-0400,MLA,"{searchbackend/recommended-products=6157, mcli...",3204901
1,PRODUCT,882352139.0,2021-08-02T23:55:51.673-0400,MLA,"{qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...",3204901
2,PRODUCT,655266729.0,2021-08-02T23:56:16.083-0400,MLA,"{qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...",3204901
3,PRODUCT,761520929.0,2021-08-02T23:56:29.989-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",3204901
4,PRODUCT,757586409.0,2021-08-02T23:56:47.887-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",3204901


In [4]:
print(df.shape)
print(df.dtypes)

(141553, 6)
event_name      object
item_id        float64
timestamp       object
site            object
experiments     object
user_id          int64
dtype: object


In [5]:
df.user_id.nunique()

7817

In [6]:
# Agregamos una variable llamada "date" que es una tranformación de la variable "timestamp" a formato fecha
df["datetime"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%dT%H:%M:%S.%f%z")
df["date"] = pd.to_datetime(df["datetime"]).dt.date
df.head(20)

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date
0,SEARCH,NaN,2021-08-02T23:55:38.966-0400,MLA,"{searchbackend/recommended-products=6157, mcli...",3204901,2021-08-02 23:55:38.966000-04:00,2021-08-02
1,PRODUCT,882352139.0,2021-08-02T23:55:51.673-0400,MLA,"{qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...",3204901,2021-08-02 23:55:51.673000-04:00,2021-08-02
2,PRODUCT,655266729.0,2021-08-02T23:56:16.083-0400,MLA,"{qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...",3204901,2021-08-02 23:56:16.083000-04:00,2021-08-02
3,PRODUCT,761520929.0,2021-08-02T23:56:29.989-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",3204901,2021-08-02 23:56:29.989000-04:00,2021-08-02
4,PRODUCT,757586409.0,2021-08-02T23:56:47.887-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",3204901,2021-08-02 23:56:47.887000-04:00,2021-08-02
5,PRODUCT,905225929.0,2021-08-02T23:56:55.898-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",3204901,2021-08-02 23:56:55.898000-04:00,2021-08-02
6,SEARCH,NaN,2021-08-02T23:46:03.785-0400,MLA,"{searchbackend/recommended-products=6157, mcli...",4484451,2021-08-02 23:46:03.785000-04:00,2021-08-02
7,SEARCH,NaN,2021-08-02T23:46:39.972-0400,MLA,"{searchbackend/recommended-products=6157, mcli...",4484451,2021-08-02 23:46:39.972000-04:00,2021-08-02
8,SEARCH,NaN,2021-08-02T23:47:13.438-0400,MLA,"{searchbackend/recommended-products=6157, mcli...",4484451,2021-08-02 23:47:13.438000-04:00,2021-08-02
9,PRODUCT,917790578.0,2021-08-02T21:08:19.146-0400,MLA,"{search/remove-ecn-tag=4954, vip/showV2V3BoxMe...",891984,2021-08-02 21:08:19.146000-04:00,2021-08-02


In [7]:
df.event_name.value_counts()

event_name
SEARCH        76750
PRODUCT       58544
CHECKOUT_1     2115
CHECKOUT_2     1786
CHECKOUT_3     1270
BUY            1088
Name: count, dtype: int64

In [8]:
df.date.value_counts()

date
2021-08-02    110459
2021-08-01     31094
Name: count, dtype: int64

In [9]:
df.groupby('date')['user_id'].nunique()

date
2021-08-01    2138
2021-08-02    6735
Name: user_id, dtype: int64

In [10]:
df.site.value_counts()

site
MLA    141553
Name: count, dtype: int64

In [11]:
df[df.user_id == 3204901].sort_values(by='datetime')

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date
63670,SEARCH,NaN,2021-08-02T12:53:11.057-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",3204901,2021-08-02 12:53:11.057000-04:00,2021-08-02
63671,SEARCH,NaN,2021-08-02T12:53:20.260-0400,MLA,"{searchbackend/recommended-products=6157, sear...",3204901,2021-08-02 12:53:20.260000-04:00,2021-08-02
28364,SEARCH,NaN,2021-08-02T16:16:05.995-0400,MLA,"{searchbackend/recommended-products=6157, sear...",3204901,2021-08-02 16:16:05.995000-04:00,2021-08-02
28365,PRODUCT,312247919.0,2021-08-02T16:16:32.788-0400,MLA,"{search/remove-ecn-tag=4954, vip/showV2V3BoxMe...",3204901,2021-08-02 16:16:32.788000-04:00,2021-08-02
28366,PRODUCT,343345729.0,2021-08-02T16:16:45.106-0400,MLA,"{search/remove-ecn-tag=4954, vip/showV2V3BoxMe...",3204901,2021-08-02 16:16:45.106000-04:00,2021-08-02
28367,PRODUCT,139306729.0,2021-08-02T16:16:53.854-0400,MLA,"{search/remove-ecn-tag=4954, vip/showV2V3BoxMe...",3204901,2021-08-02 16:16:53.854000-04:00,2021-08-02
28368,SEARCH,NaN,2021-08-02T16:17:15.174-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",3204901,2021-08-02 16:17:15.174000-04:00,2021-08-02
28369,SEARCH,NaN,2021-08-02T16:17:27.593-0400,MLA,"{searchbackend/recommended-products=6157, sear...",3204901,2021-08-02 16:17:27.593000-04:00,2021-08-02
28370,SEARCH,NaN,2021-08-02T16:18:43.979-0400,MLA,"{searchbackend/recommended-products=6157, sear...",3204901,2021-08-02 16:18:43.979000-04:00,2021-08-02
28371,PRODUCT,577946929.0,2021-08-02T16:19:00.111-0400,MLA,"{search/remove-ecn-tag=4954, vip/showV2V3BoxMe...",3204901,2021-08-02 16:19:00.111000-04:00,2021-08-02


In [12]:
df[df.event_name == "BUY"].head()

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date
204,BUY,740440936.0,2021-08-02T23:53:46.363-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",426144,2021-08-02 23:53:46.363000-04:00,2021-08-02
295,BUY,680029039.0,2021-08-02T23:38:37.024-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:38:37.024000-04:00,2021-08-02
305,BUY,988145209.0,2021-08-02T23:41:31.971-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:41:31.971000-04:00,2021-08-02
447,BUY,885565729.0,2021-08-02T23:11:25.388-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",2866832,2021-08-02 23:11:25.388000-04:00,2021-08-02
562,BUY,934373429.0,2021-08-02T20:01:39.802-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",411939,2021-08-02 20:01:39.802000-04:00,2021-08-02


In [13]:
df[df.user_id == 426144].sort_values(by='datetime')

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date
138286,PRODUCT,601100429.0,2021-08-01T16:04:44.959-0400,MLA,"{search/remove-ecn-tag=4954, vip/showV2V3BoxMe...",426144,2021-08-01 16:04:44.959000-04:00,2021-08-01
1546,PRODUCT,968903688.0,2021-08-02T23:41:23.828-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",426144,2021-08-02 23:41:23.828000-04:00,2021-08-02
1547,SEARCH,NaN,2021-08-02T23:41:49.278-0400,MLA,"{searchbackend/recommended-products=6158, sear...",426144,2021-08-02 23:41:49.278000-04:00,2021-08-02
1548,SEARCH,NaN,2021-08-02T23:41:55.449-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",426144,2021-08-02 23:41:55.449000-04:00,2021-08-02
1549,SEARCH,NaN,2021-08-02T23:43:31.849-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",426144,2021-08-02 23:43:31.849000-04:00,2021-08-02
1550,SEARCH,NaN,2021-08-02T23:44:15.814-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",426144,2021-08-02 23:44:15.814000-04:00,2021-08-02
1551,SEARCH,NaN,2021-08-02T23:45:27.552-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",426144,2021-08-02 23:45:27.552000-04:00,2021-08-02
1552,PRODUCT,56814729.0,2021-08-02T23:45:38.768-0400,MLA,"{qadb/sa-on-vip=6696, vip/showV2V3BoxMessages=...",426144,2021-08-02 23:45:38.768000-04:00,2021-08-02
1553,SEARCH,NaN,2021-08-02T23:45:45.007-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",426144,2021-08-02 23:45:45.007000-04:00,2021-08-02
1554,PRODUCT,441857729.0,2021-08-02T23:45:55.646-0400,MLA,"{search/remove-ecn-tag=4954, qadb/sa-on-vip=66...",426144,2021-08-02 23:45:55.646000-04:00,2021-08-02


In [14]:
print(df[df.user_id == 426144].experiments[200].strip('{}').split(', '))
print(df[df.user_id == 426144].experiments[201].strip('{}').split(', '))
print(df[df.user_id == 426144].experiments[202].strip('{}').split(', '))
print(df[df.user_id == 426144].experiments[203].strip('{}').split(', '))
print(df[df.user_id == 426144].experiments[204].strip('{}').split(', '))

['search/remove-ecn-tag=4954', 'searchbackend/recommended-products=6158', 'searchbackend/cbt-antiboost=6333', 'search/back-filters=5059', 'filters/sort-by-ranking=6971', 'searchbackend/seller-reputation-change=4552', 'cookiesConsentBanner=DEFAULT', 'frontend/assetsCdnDomainMLA=DEFAULT', 'frontend/assetsCdnDomainMLU=DEFAULT', 'searchbackend/item-reputation=3824']
['buyingflow/user-track=6796', 'buyingflow/escWebMLA=2874']
['buyingflow/user-track=6796', 'buyingflow/address_hub=3574']
['buyingflow/user-track=6796', 'buyingflow/address_hub=3574']
['buyingflow/user-track=6796', 'buyingflow/address_hub=3574']


In [15]:
df[df.user_id == 426144].experiments[200]

'{search/remove-ecn-tag=4954, searchbackend/recommended-products=6158, searchbackend/cbt-antiboost=6333, search/back-filters=5059, filters/sort-by-ranking=6971, searchbackend/seller-reputation-change=4552, cookiesConsentBanner=DEFAULT, frontend/assetsCdnDomainMLA=DEFAULT, frontend/assetsCdnDomainMLU=DEFAULT, searchbackend/item-reputation=3824}'

In [16]:
## Search
df[df.user_id == 426144].experiments[200].strip('{}').split(', ')

['search/remove-ecn-tag=4954',
 'searchbackend/recommended-products=6158',
 'searchbackend/cbt-antiboost=6333',
 'search/back-filters=5059',
 'filters/sort-by-ranking=6971',
 'searchbackend/seller-reputation-change=4552',
 'cookiesConsentBanner=DEFAULT',
 'frontend/assetsCdnDomainMLA=DEFAULT',
 'frontend/assetsCdnDomainMLU=DEFAULT',
 'searchbackend/item-reputation=3824']

In [17]:
## Product
df[df.user_id == 426144].experiments[1566].strip('{}').split(', ')

['search/remove-ecn-tag=4954',
 'qadb/sa-on-vip=6696',
 'vip/showV2V3BoxMessages=6430',
 'pdp/viewItemPageMigrationDesktopReviewsNoTabs=4856',
 'cookiesConsentBanner=DEFAULT',
 'vip/shippingCalculatorMigrationModalExperiment=6551',
 'vip/carousel-v2p-above-the-fold=6786',
 'vip/seller-with-tooltip=4692']

In [18]:
df[df.user_id == 6498883].sort_values(by='datetime')

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date
131034,SEARCH,NaN,2021-08-01T20:47:19.528-0400,MLA,"{searchbackend/recommended-products=6157, sear...",6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01
131043,SEARCH,NaN,2021-08-01T20:54:02.302-0400,MLA,"{searchbackend/recommended-products=6157, sear...",6498883,2021-08-01 20:54:02.302000-04:00,2021-08-01
292,CHECKOUT_1,680029039.0,2021-08-02T23:37:16.248-0400,MLA,"{buyingflow/user-track=6796, buyingflow/escWeb...",6498883,2021-08-02 23:37:16.248000-04:00,2021-08-02
293,CHECKOUT_2,680029039.0,2021-08-02T23:37:28.379-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:37:28.379000-04:00,2021-08-02
294,CHECKOUT_3,680029039.0,2021-08-02T23:38:14.079-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:38:14.079000-04:00,2021-08-02
295,BUY,680029039.0,2021-08-02T23:38:37.024-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:38:37.024000-04:00,2021-08-02
296,CHECKOUT_2,680029039.0,2021-08-02T23:39:08.456-0400,MLA,{buyingflow/user-track=6796},6498883,2021-08-02 23:39:08.456000-04:00,2021-08-02
297,CHECKOUT_2,680029039.0,2021-08-02T23:39:38.513-0400,MLA,{buyingflow/user-track=6796},6498883,2021-08-02 23:39:38.513000-04:00,2021-08-02
298,CHECKOUT_2,680029039.0,2021-08-02T23:39:42.170-0400,MLA,{buyingflow/user-track=6796},6498883,2021-08-02 23:39:42.170000-04:00,2021-08-02
299,SEARCH,NaN,2021-08-02T23:39:55.540-0400,MLA,"{searchbackend/recommended-products=6157, sear...",6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02


In [19]:
df[df.user_id == 466].sort_values(by='datetime')

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date
22250,SEARCH,NaN,2021-08-02T19:44:09.487-0400,MLA,"{searchbackend/recommended-products=6158, mcli...",466,2021-08-02 19:44:09.487000-04:00,2021-08-02
22251,SEARCH,NaN,2021-08-02T19:44:16.693-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",466,2021-08-02 19:44:16.693000-04:00,2021-08-02
22252,SEARCH,NaN,2021-08-02T19:44:23.557-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",466,2021-08-02 19:44:23.557000-04:00,2021-08-02
22253,SEARCH,NaN,2021-08-02T19:44:30.785-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",466,2021-08-02 19:44:30.785000-04:00,2021-08-02
22254,SEARCH,NaN,2021-08-02T19:45:00.518-0400,MLA,"{search/remove-ecn-tag=4954, searchbackend/rec...",466,2021-08-02 19:45:00.518000-04:00,2021-08-02
22255,PRODUCT,831899129.0,2021-08-02T19:45:08.146-0400,MLA,"{vip/showV2V3BoxMessages=6430, search/tendency...",466,2021-08-02 19:45:08.146000-04:00,2021-08-02
22257,PRODUCT,789004239.0,2021-08-02T19:45:44.269-0400,MLA,"{search/remove-ecn-tag=4954, vip/showV2V3BoxMe...",466,2021-08-02 19:45:44.269000-04:00,2021-08-02
22260,SEARCH,NaN,2021-08-02T19:47:55.404-0400,MLA,"{searchbackend/recommended-products=6158, sear...",466,2021-08-02 19:47:55.404000-04:00,2021-08-02
22261,PRODUCT,329016619.0,2021-08-02T19:48:01.879-0400,MLA,"{search/remove-ecn-tag=4954, vip/showV2V3BoxMe...",466,2021-08-02 19:48:01.879000-04:00,2021-08-02
22262,PRODUCT,407593419.0,2021-08-02T19:48:17.168-0400,MLA,"{search/remove-ecn-tag=4954, vip/showV2V3BoxMe...",466,2021-08-02 19:48:17.168000-04:00,2021-08-02


In [20]:
# Find all item_id and user_id pairs that end with a 'BUY' event
buy_pairs = df[df['event_name'] == 'BUY'][['item_id', 'user_id']].drop_duplicates()

# Define a set for quick lookup
buy_pairs_set = set(tuple(x) for x in buy_pairs.values)

# Function to determine if the row is in the purchase funnel
def in_purchase_funnel(row):
    if pd.notna(row['item_id']):  # Only consider rows with a valid item_id
        return 1 if (row['item_id'], row['user_id']) in buy_pairs_set else 0
    return 0

# Apply the function to create the purchase_funnel_flag column
df['purchase_funnel_flag'] = df.apply(in_purchase_funnel, axis=1)

Creamos entonces una variable **purchase_funnel_flag**, que nos permita asignar 1 a todos aquellos experimentos durante la cadena de compra por los cuales paso nuestro cliente, llegando así a una **compra exitosa**

In [21]:
# Sort the DataFrame by user_id and datetime
df_sorted = df.sort_values(by=['user_id', 'datetime']).reset_index(drop=True)

# Create a mask for rows where purchase_funnel_flag is 1
mask = df_sorted['purchase_funnel_flag'] == 1

# Create a shifted version of the event_name column
df_sorted['prev_event'] = df_sorted['event_name'].shift(1)
df_sorted['prev_user'] = df_sorted['user_id'].shift(1).fillna(0).astype(int)
df_sorted['prev_date'] = df_sorted['date'].shift(1)

# Create a mask for rows where the previous event is SEARCH and user_id matches
search_mask = (df_sorted['prev_event'] == 'SEARCH') & (df_sorted['user_id'] == df_sorted['prev_user']) & (df_sorted['date'] == df_sorted['prev_date']) & (df_sorted['purchase_funnel_flag'] == 1)
search_mask_1 = search_mask.shift(-1)

# Update purchase_funnel_flag where mask is True and search_mask is True
df_sorted.loc[mask | search_mask_1, 'purchase_funnel_flag'] = 1

# Drop the 'prev_event' column
df_sorted.drop(columns=['prev_event','prev_user','prev_date'], inplace=True)

In [22]:
#df_sorted[df_sorted.user_id == 466].sort_values(by='datetime')
df_sorted[df_sorted.user_id == 6498883].sort_values(by='datetime')

,event_name,item_id,timestamp,site,experiments,user_id,datetime,date,purchase_funnel_flag
117111,SEARCH,NaN,2021-08-01T20:47:19.528-0400,MLA,"{searchbackend/recommended-products=6157, sear...",6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0
117112,SEARCH,NaN,2021-08-01T20:54:02.302-0400,MLA,"{searchbackend/recommended-products=6157, sear...",6498883,2021-08-01 20:54:02.302000-04:00,2021-08-01,0
117113,CHECKOUT_1,680029039.0,2021-08-02T23:37:16.248-0400,MLA,"{buyingflow/user-track=6796, buyingflow/escWeb...",6498883,2021-08-02 23:37:16.248000-04:00,2021-08-02,1
117114,CHECKOUT_2,680029039.0,2021-08-02T23:37:28.379-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:37:28.379000-04:00,2021-08-02,1
117115,CHECKOUT_3,680029039.0,2021-08-02T23:38:14.079-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:38:14.079000-04:00,2021-08-02,1
117116,BUY,680029039.0,2021-08-02T23:38:37.024-0400,MLA,"{buyingflow/user-track=6796, buyingflow/addres...",6498883,2021-08-02 23:38:37.024000-04:00,2021-08-02,1
117117,CHECKOUT_2,680029039.0,2021-08-02T23:39:08.456-0400,MLA,{buyingflow/user-track=6796},6498883,2021-08-02 23:39:08.456000-04:00,2021-08-02,1
117118,CHECKOUT_2,680029039.0,2021-08-02T23:39:38.513-0400,MLA,{buyingflow/user-track=6796},6498883,2021-08-02 23:39:38.513000-04:00,2021-08-02,1
117119,CHECKOUT_2,680029039.0,2021-08-02T23:39:42.170-0400,MLA,{buyingflow/user-track=6796},6498883,2021-08-02 23:39:42.170000-04:00,2021-08-02,1
117120,SEARCH,NaN,2021-08-02T23:39:55.540-0400,MLA,"{searchbackend/recommended-products=6157, sear...",6498883,2021-08-02 23:39:55.540000-04:00,2021-08-02,1


In [23]:
print(df.purchase_funnel_flag.value_counts())
print(df_sorted.purchase_funnel_flag.value_counts())

purchase_funnel_flag
0    134867
1      6686
Name: count, dtype: int64
purchase_funnel_flag
0    134068
1      7485
Name: count, dtype: int64


Procesamiento de la información, inicialmente convertimos la columna **experiments** a diccionarios, y luego realizamos la apertira de cada uno de los mismos para obener por fila cada uno de los eventos,y limpieza de columnas que ya no utilizaremos para las finalidades del ejerccio

In [24]:
# Apply the function to the 'experiments' column and create a new column with the dictionaries
df_sorted['experiments_dict'] = df_sorted['experiments'].apply(utilities.str_to_dict)

# aplicamos funcion explode_dict para apertura de diccionarios
df_exploded = df_sorted.apply(utilities.explode_dict, axis=1).reset_index(drop=True)
df_exploded = pd.concat(df_exploded.to_list(), ignore_index=True)

# Drop the 'experiments_dict' column as it is no longer needed
df_exploded.drop(columns=['experiments_dict','timestamp','experiments'], inplace=True)

# separamos columna experiment, para obtener su correspondiente descripción y variante
df_exploded[['path', 'experiment']] = df_exploded['experiment_key'].str.split('/', expand=True)
df_exploded.rename(columns={'experiment_value': 'variant'}, inplace=True)

In [25]:
#df_exploded[(df_exploded.user_id == 466)&(df_exploded.experiment == 'search-list-algorithms')].sort_values(by='datetime')
df_exploded[(df_exploded.user_id == 466)].sort_values(by='datetime')

,event_name,item_id,site,user_id,datetime,date,purchase_funnel_flag,experiment_key,variant,path,experiment
1766,SEARCH,NaN,MLA,466,2021-08-02 19:44:09.487000-04:00,2021-08-02,0,searchbackend/recommended-products,6158,searchbackend,recommended-products
1782,SEARCH,NaN,MLA,466,2021-08-02 19:44:09.487000-04:00,2021-08-02,0,mclics/show-pads-global,5176,mclics,show-pads-global
1781,SEARCH,NaN,MLA,466,2021-08-02 19:44:09.487000-04:00,2021-08-02,0,search/best-seller-fail-fast-edition-MLA,4916,search,best-seller-fail-fast-edition-MLA
1780,SEARCH,NaN,MLA,466,2021-08-02 19:44:09.487000-04:00,2021-08-02,0,frontend/assetsCdnDomainMLA,DEFAULT,frontend,assetsCdnDomainMLA
1779,SEARCH,NaN,MLA,466,2021-08-02 19:44:09.487000-04:00,2021-08-02,0,cookiesConsentBanner,DEFAULT,cookiesConsentBanner,None
1778,SEARCH,NaN,MLA,466,2021-08-02 19:44:09.487000-04:00,2021-08-02,0,searchbackend/seller-reputation-change,4552,searchbackend,seller-reputation-change
1777,SEARCH,NaN,MLA,466,2021-08-02 19:44:09.487000-04:00,2021-08-02,0,mclics/show-pads-search-list,5146,mclics,show-pads-search-list
1775,SEARCH,NaN,MLA,466,2021-08-02 19:44:09.487000-04:00,2021-08-02,0,search/remove-ecn-tag,4954,search,remove-ecn-tag
1776,SEARCH,NaN,MLA,466,2021-08-02 19:44:09.487000-04:00,2021-08-02,0,mclics/search-pads-none-desktop-mla,3478,mclics,search-pads-none-desktop-mla
1773,SEARCH,NaN,MLA,466,2021-08-02 19:44:09.487000-04:00,2021-08-02,0,frontend/assetsCdnDomainMLU,DEFAULT,frontend,assetsCdnDomainMLU


### 1. **Agrupación de la información para realizar el conteo**

In [26]:
df_exploded['item_id'] = df_exploded['item_id'].fillna(0)

# Agrupamos con variables necesarias para conteo
df_grouped = df_exploded.groupby(['date','experiment','variant','user_id','path']).agg({'purchase_funnel_flag': 'max',  'item_id': lambda x: x[df_exploded['purchase_funnel_flag'] != 0].nunique() }).reset_index()

df_grouped.head()

,date,experiment,variant,user_id,path,purchase_funnel_flag,item_id
0,2021-08-01,HideTransitionModal,6646,336575,mshops,0,0
1,2021-08-01,HideTransitionModal,6646,3677574,mshops,0,0
2,2021-08-01,HideTransitionModal,6647,2234611,mshops,0,0
3,2021-08-01,HideTransitionModal,6647,3023131,mshops,0,0
4,2021-08-01,HideTransitionModal,6647,9116431,mshops,0,0


QUe hacer cuando un cliente compra varias veces el mismo artículo ?

In [27]:
df_exploded[df_exploded.user_id == 6498883].sort_values(by='datetime').head(50)
#df_exploded[df_exploded.user_id == 466].sort_values(by='datetime')


,event_name,item_id,site,user_id,datetime,date,purchase_funnel_flag,experiment_key,variant,path,experiment
1107071,SEARCH,0.0,MLA,6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0,searchbackend/recommended-products,6157,searchbackend,recommended-products
1107088,SEARCH,0.0,MLA,6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0,mclics/show-pads-global,5176,mclics,show-pads-global
1107087,SEARCH,0.0,MLA,6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0,search/best-seller-fail-fast-edition-MLA,4916,search,best-seller-fail-fast-edition-MLA
1107086,SEARCH,0.0,MLA,6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0,frontend/assetsCdnDomainMLA,DEFAULT,frontend,assetsCdnDomainMLA
1107085,SEARCH,0.0,MLA,6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0,cookiesConsentBanner,DEFAULT,cookiesConsentBanner,None
1107083,SEARCH,0.0,MLA,6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0,mclics/show-pads-search-list,5146,mclics,show-pads-search-list
1107082,SEARCH,0.0,MLA,6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0,mclics/search-pads-none-desktop-mla,3478,mclics,search-pads-none-desktop-mla
1107081,SEARCH,0.0,MLA,6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0,search/remove-ecn-tag,4954,search,remove-ecn-tag
1107080,SEARCH,0.0,MLA,6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0,searchbackend/item-reputation,3824,searchbackend,item-reputation
1107084,SEARCH,0.0,MLA,6498883,2021-08-01 20:47:19.528000-04:00,2021-08-01,0,searchbackend/seller-reputation-change,4553,searchbackend,seller-reputation-change


In [28]:
#df_grouped[df_grouped.user_id == 466].sort_values(by='experiment')
df_grouped[df_grouped.user_id == 6498883].sort_values(by='date').head(20)

,date,experiment,variant,user_id,path,purchase_funnel_flag,item_id
1162,2021-08-01,ads-adsearch-boost-incremental-desktop-mla,3809,6498883,mclics,0,0
33297,2021-08-01,ungroup-products,3273,6498883,search,0,0
31845,2021-08-01,sort-by-ranking,7057,6498883,filters,0,0
28337,2021-08-01,show-pads-search-list,5146,6498883,mclics,0,0
22942,2021-08-01,seller-reputation-change,4553,6498883,searchbackend,0,0
21186,2021-08-01,search-pads-none-desktop-mla,3478,6498883,mclics,0,0
20018,2021-08-01,search-list-algorithms,5528,6498883,mclics,0,0
18052,2021-08-01,remove-ecn-tag,4954,6498883,search,0,0
27144,2021-08-01,show-pads-global,5176,6498883,mclics,0,0
14276,2021-08-01,item-reputation,3824,6498883,searchbackend,0,0


In [29]:
df_grouped_00 = df_grouped.groupby(['date', 'experiment', 'variant']).agg({'user_id': 'count', 'purchase_funnel_flag': 'sum'}).reset_index()
df_grouped_00 = df_grouped_00.rename(columns={"user_id": "participants" , "purchase_funnel_flag": "purchases"})
df_grouped_00['buy_rate'] = df_grouped_00 ['purchases']/df_grouped_00 ['participants']
df_grouped_00['buy_rate_percent'] = df_grouped_00['buy_rate'] .apply(lambda x: '{:.2f}%'.format(x*100))
#df_5 = df_5.sort_values(['date','experiment_name', 'conversion_rate_%', ], ascending = [True, True, False], ignore_index=True)
#df_5


In [30]:
df_grouped_00.shape

(143, 7)

In [31]:
df_grouped_00.head(30)

,date,experiment,variant,participants,purchases,buy_rate,buy_rate_percent
0,2021-08-01,HideTransitionModal,6646,2,0,0.000000,0.00%
1,2021-08-01,HideTransitionModal,6647,3,0,0.000000,0.00%
2,2021-08-01,HideTransitionModal,6993,5,0,0.000000,0.00%
3,2021-08-01,HideTransitionModal,DEFAULT,12,0,0.000000,0.00%
4,2021-08-01,address_hub,3574,189,143,0.756614,75.66%
5,2021-08-01,ads-adsearch-boost-incremental-desktop-mla,3809,1211,52,0.042940,4.29%
6,2021-08-01,assetsCdnDomainMLA,DEFAULT,1766,89,0.050396,5.04%
7,2021-08-01,assetsCdnDomainMLU,DEFAULT,1764,89,0.050454,5.05%
8,2021-08-01,back-filters,5059,1764,89,0.050454,5.05%
9,2021-08-01,back-filters,DEFAULT,1,0,0.000000,0.00%


In [32]:
df_grouped_00[df_grouped_00.experiment=='checkOnBehavior'].head(2)

,date,experiment,variant,participants,purchases,buy_rate,buy_rate_percent
19,2021-08-01,checkOnBehavior,348,22,0,0.000000,0.00%
20,2021-08-01,checkOnBehavior,349,21,1,0.047619,4.76%


In [33]:
df_grouped_00.groupby('date')['participants'].max()

date
2021-08-01    1845
2021-08-02    5962
Name: participants, dtype: int64

In [40]:
df_grouped_00.date = df_grouped_00.date.astype(str)

In [84]:
df_grouped_00.to_csv('/Users/juanmanuelpaiba/Documents/Juan_Paiba/AB_Testing_Meli/python_scripts/grouped_inf.csv', sep = ",", index = False)

In [88]:
archivo_valido = not df_results.empty

In [89]:
archivo_valido

True

* validacion de poblacion entre dias / agrupar por experimentos cant usuarios por dias y calcular porcentaje, exposicion consistentes dia1 vs dia 2
* experimento almenos dos variantes / experimentos no tengan mucho default/ no se asigno variante / cant defaults
* cuantos usuarios/ cadena de navegacion termino el dia siguiente

In [44]:
url = "https://raw.githubusercontent.com/jmpaiba123/AB_Testing_Meli/main/data/Outputs/grouped_inf.csv"
df_results = pd.read_csv(url, sep=",")
print(df_results.head())

         date           experiment  variant  participants  purchases  buy_rate buy_rate_percent
0  2021-08-01  HideTransitionModal     6646             2          0  0.000000            0.00%
1  2021-08-01  HideTransitionModal     6647             3          0  0.000000            0.00%
2  2021-08-01  HideTransitionModal     6993             5          0  0.000000            0.00%
3  2021-08-01  HideTransitionModal  DEFAULT            12          0  0.000000            0.00%
4  2021-08-01          address_hub     3574           189        143  0.756614           75.66%


In [36]:
# Filtramos los datos del dataframe para obtener los resultados del día y experimento que deseamos conocer
df_grouped_00[(df_grouped_00['date'] == '2021-08-01') & (df_grouped_00['experiment'] == 'checkOnBehavior')]

,date,experiment,variant,participants,purchases,buy_rate,buy_rate_percent


In [81]:
#pip install flask
df_grouped_00.to_csv('grouped_inf.csv', sep = ",", index = False)

In [80]:
import os
os.getcwd()

'/Users/juanmanuelpaiba/Documents/Juan_Paiba/AB_Testing_Meli/jupyter_notebooks'

In [82]:
df_results = pd.read_csv("grouped_inf.csv", sep=",")

In [39]:
app = Flask(__name__)

@app.route('/experiment/<experiment_id>/result')

def get_experiment_results(experiment_id):

    return jsonify({'resultados': f"mensaje + {experiment_id}"})

# Iniicamos la API en el local host
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [64]:
filtered_results = df_results[(df_results['experiment'] == "HideTransitionModal")]

In [65]:
filtered_results

,date,experiment,variant,participants,purchases,buy_rate,buy_rate_percent
0,2021-08-01,HideTransitionModal,6646,2,0,0.000000,0.00%
1,2021-08-01,HideTransitionModal,6647,3,0,0.000000,0.00%
2,2021-08-01,HideTransitionModal,6993,5,0,0.000000,0.00%
3,2021-08-01,HideTransitionModal,DEFAULT,12,0,0.000000,0.00%
71,2021-08-02,HideTransitionModal,6646,17,1,0.058824,5.88%
72,2021-08-02,HideTransitionModal,6647,16,0,0.000000,0.00%
73,2021-08-02,HideTransitionModal,6993,16,1,0.062500,6.25%
74,2021-08-02,HideTransitionModal,DEFAULT,31,0,0.000000,0.00%


In [47]:
df_results.head()

,date,experiment,variant,participants,purchases,buy_rate,buy_rate_percent
0,2021-08-01,HideTransitionModal,6646,2,0,0.000000,0.00%
1,2021-08-01,HideTransitionModal,6647,3,0,0.000000,0.00%
2,2021-08-01,HideTransitionModal,6993,5,0,0.000000,0.00%
3,2021-08-01,HideTransitionModal,DEFAULT,12,0,0.000000,0.00%
4,2021-08-01,address_hub,3574,189,143,0.756614,75.66%


In [77]:
import os
import pandas as pd
from flask import (Flask, redirect, jsonify, render_template, request,
                   send_from_directory, url_for)

url = "./grouped_inf.csv"

try:
    df_results = pd.read_csv(url, sep=",")
except ValueError:
    print("error en la lectura del archivo") 


app = Flask(__name__)

@app.route('/experiment/<exp_name>/result')

def get_experiment_results(exp_name):

    # Obtenemos el valor del parámetro "day" para la solicitud HTTP
    day = request.args.get('day')
    print(day)
    print(exp_name)

    # Validamos que la fecha proporcionada es correcta, de lo contrario mostrara el error "fecha invalida"
    if len(day) == 0:
        return jsonify({'error': 'fecha invalida'}), 400
    try:
        day_validation = pd.Timestamp(day).floor('D')
    except ValueError:
        return jsonify({'error': 'fecha invalida'}), 400
    
    # Filtramos los datos del dataframe para obtener los resultados del día y experimento que deseamos conocer
    filtered_results = df_results[(df_results['experiment'] == exp_name)&(df_results['date'] == day)]
    print(filtered_results)
    # Si no hay resultados para ese día y ese experimento, se muestra el error "experimento no encontrado"
    if filtered_results.empty:
        return jsonify({'error': 'experimento no encontrado'}), 404
    
    # Calculamos los resultados requeridos, en este caso, total de participantes y total de compras
    total_participants = filtered_results['participants'].sum()
    winners = filtered_results.loc[filtered_results['purchases'].idxmax(), 'variant']
    variants = []
    for index, row in filtered_results.iterrows():
        variants.append({
            'id': row['variant'],
            'numero_de_compras': str(row['purchases'])
        })
    
    # Presentamos los resultados en el formato requerido
    results = {
        exp_name: {
            'numero_de_participantes': str(total_participants),
            'ganador': winners,
            'variantes': variants
        }
    }
    return jsonify({'resultados': results})

# Iniicamos la API en el local host
if __name__ == '__main__':
    app.run()

FileNotFoundError: [Errno 2] No such file or directory: './grouped_inf.csv'